In [1]:
import sys
sys.path.insert(0, 'scripts/')

import resnet50
import lasagne
import theano
import os
import skimage.transform
import skimage.io
import pickle

import theano.tensor as T
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

-----

### Resnet-50

In [4]:
# Download pretrained weights from:
# https://s3.amazonaws.com/lasagne/recipes/pretrained/imagenet/resnet50.pkl
model = pickle.load(open('data/resnet50.pkl', 'rb'))
classes = model['synset_words']
mean_image = model['mean_image']

In [5]:
def cat_dog(classes):
    cat_classes = [i for i,x in enumerate(classes)\
                   if ' cat' in x\
                   or 'lue point Siamese' in x\
                   or 'tabby, queen' in x\
                   or 'mouser' in x
                  ]
    dog_classes = [i for i,x in enumerate(classes)\
               if ' dog' in x or ' pointer' in x or 'setter' in x or 'spaniel' in x or 'griffon' in x\
               or 'retriever' in x or 'terrier' in x or 'schnauzer' in x or 'courser' in x\
               or 'hound' in x or 'Weimaraner' in x or 'beagle' in x or 'harrier' in x\
               or 'bluetick' in x or 'redbone' in x or 'rhodesian ridgeback' in x or 'dalmatian' in x\
               or 'ur, mongrel, mutt' in x or 'eist, fice' in x or 'corgi' in x or 'exican hairless' in x\
               or 'poodle' in x or 'asenji' in x or 'eonberg' in x or 'pitz' in x or 'Pomeranian' in x\
               or 'eeshond' in x or 'how, chow chow' in x or 'amoyed, Samoyede' in x or 'reat Pyrenees' in x\
               or 'ekinese, Pekingese, Peke' in x or 'hih-Tzu' in x or 'hihuahua' in x or 'puppy' in x
              ]
    return cat_classes, dog_classes
cat_classes, dog_classes = cat_dog(classes)

In [ ]:
#computational graph construction
net = resnet50.build_model()

#load weights
lasagne.layers.set_all_param_values(net['prob'], model['values'])

#define placeholder variables
input_var = T.tensor4('input_var')
output_var = lasagne.layers.get_output(net['prob'], {net['input']: input_var}, deterministic=True)

#compile prediction function
predict_fn = theano.function([input_var], output_var)

In [ ]:
def preprocess(img):
    if len(img.shape) == 2:
        return np.array([np.repeat(np.array(img), 3).reshape(3,224,-1)])
    return np.array([np.swapaxes(np.swapaxes(img, 0,1), 0,2)])

In [ ]:
from os import listdir

path = 'data/test/cropped_samples224/'

with open('resnet_max.csv', 'a+') as f:
    f.write('file,label\n')
i = 0

for filename in listdir(path):
    img = skimage.io.imread(path+filename)
    prob = predict_fn(preprocess(img)).ravel()
    cat_proba = np.mean(np.sort(prob[cat_classes])[-3:])
    dog_proba = np.mean(np.sort(prob[dog_classes])[-3:])
    with open('resnet_max.csv', 'a+') as f:
        if cat_proba > dog_proba:
            f.write(filename+',cat\n')
        else:
            f.write(filename+',dog\n')

    i+=1
    if i % 100 == 0:
        print(i)

----- 

### Overfeat

In [6]:
overfeat_classes = np.load('data/overfeat_classes.npy')
cat_classes, dog_classes = cat_dog(overfeat_classes)

In [ ]:
import subprocess

path = 'data/test/cropped_samples224/'

with open('overfeat_max.csv', 'a+') as f:
    f.write('file,label\n')
i = 0

for filename in listdir(path):
    
    res = subprocess.check_output(" overfeat/bin/macos/overfeat -n 100 -l "+path+filename,stderr=subprocess.STDOUT,shell=True)
    names = []
    probas = []
    for r in res.split('\n'):
        if len(r) > 0:
            parse = r.split(' ')
            names.append(' '.join(parse[:-1]))
            probas.append(float(parse[-1]))
    probas = np.array(probas)
    
    cats_proba, dogs_proba = cat_dog(names)
    cats_proba = np.nan_to_num(np.sum(probas[cats_proba][:3])/3.)
    dogs_proba = np.nan_to_num(np.sum(probas[dogs_proba][:3])/3.)
    with open('overfeat_max.csv', 'a+') as f:
        if cats_proba > dogs_proba:
            f.write(filename+',cat\n')
        elif cats_proba < dogs_proba:
            f.write(filename+',dog\n')
        elif np.random.rand() > 0.5:
            f.write(filename+',cat\n')
        else:
            f.write(filename+',dog\n')

    i+=1
    if i % 10 == 0:
        print i,

-----

### VGG-16

In [7]:
# copyright: see http://www.robots.ox.ac.uk/~vgg/research/very_deep/
from lasagne.layers import InputLayer
from lasagne.layers import DenseLayer
from lasagne.layers import NonlinearityLayer
from lasagne.layers import DropoutLayer
from lasagne.layers import Pool2DLayer as PoolLayer
from lasagne.layers import Conv2DLayer as ConvLayer
from lasagne.nonlinearities import softmax

def build_model():
    net = {}
    net['input'] = InputLayer((None, 3, 224, 224))
    net['conv1_1'] = ConvLayer(
        net['input'], 64, 3, pad=1, flip_filters=False)
    net['conv1_2'] = ConvLayer(
        net['conv1_1'], 64, 3, pad=1, flip_filters=False)
    net['pool1'] = PoolLayer(net['conv1_2'], 2)
    net['conv2_1'] = ConvLayer(
        net['pool1'], 128, 3, pad=1, flip_filters=False)
    net['conv2_2'] = ConvLayer(
        net['conv2_1'], 128, 3, pad=1, flip_filters=False)
    net['pool2'] = PoolLayer(net['conv2_2'], 2)
    net['conv3_1'] = ConvLayer(
        net['pool2'], 256, 3, pad=1, flip_filters=False)
    net['conv3_2'] = ConvLayer(
        net['conv3_1'], 256, 3, pad=1, flip_filters=False)
    net['conv3_3'] = ConvLayer(
        net['conv3_2'], 256, 3, pad=1, flip_filters=False)
    net['pool3'] = PoolLayer(net['conv3_3'], 2)
    net['conv4_1'] = ConvLayer(
        net['pool3'], 512, 3, pad=1, flip_filters=False)
    net['conv4_2'] = ConvLayer(
        net['conv4_1'], 512, 3, pad=1, flip_filters=False)
    net['conv4_3'] = ConvLayer(
        net['conv4_2'], 512, 3, pad=1, flip_filters=False)
    net['pool4'] = PoolLayer(net['conv4_3'], 2)
    net['conv5_1'] = ConvLayer(
        net['pool4'], 512, 3, pad=1, flip_filters=False)
    net['conv5_2'] = ConvLayer(
        net['conv5_1'], 512, 3, pad=1, flip_filters=False)
    net['conv5_3'] = ConvLayer(
        net['conv5_2'], 512, 3, pad=1, flip_filters=False)
    net['pool5'] = PoolLayer(net['conv5_3'], 2)
    net['fc6'] = DenseLayer(net['pool5'], num_units=4096)
    net['fc6_dropout'] = DropoutLayer(net['fc6'], p=0.5)
    net['fc7'] = DenseLayer(net['fc6_dropout'], num_units=4096)
    net['fc7_dropout'] = DropoutLayer(net['fc7'], p=0.5)
    net['fc8'] = DenseLayer(
        net['fc7_dropout'], num_units=1000, nonlinearity=None)
    net['prob'] = NonlinearityLayer(net['fc8'], softmax)

    return net

net = build_model()
with open('data/vgg16.pkl') as f:
    weights = pickle.load(f)
lasagne.layers.set_all_param_values(net["prob"],weights['param values'])

In [ ]:
input_image = T.tensor4('input')
output = lasagne.layers.get_output(net['prob'], input_image,deterministic=True)
predict_fn = theano.function([input_image], output) 

In [ ]:
classes = pickle.load(open('vgg_classes.pkl'))
cat_classes, dog_classes = cat_dog(classes)

path = 'data/test/cropped_samples224/'

with open('vgg_max.csv', 'a+') as f:
    f.write('file,label\n')
i = 0

for filename in listdir(path):
    img = skimage.io.imread(path+filename)
    prob = predict_fn(preprocess(img)).ravel()
    cat_proba = np.mean(np.sort(prob[cat_classes])[-3:])
    dog_proba = np.mean(np.sort(prob[dog_classes])[-3:])
    with open('vgg_max.csv', 'a+') as f:
        if cat_proba > dog_proba:
            f.write(filename+',cat\n')
        else:
            f.write(filename+',dog\n')

    i+=1
    if i % 100 == 0:
        print(i)